# Capstone Mushroom Classification 

The 

First we use beautifulSoup to scrape the website "svampguiden.com" to get the names of the most common mushrooms in Sweden. We will save these in a dictionary together with a label describing if they are edible or not.  

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
# Specify url of mushroom website
mushroom_url = 'http://svampguiden.com/databasen/lista/'

page = urlopen(mushroom_url)
soup = BeautifulSoup(page, 'html.parser')


In [5]:
latin_html = soup.find_all(class_="vnamn")

In [6]:

for idx, name in enumerate(latin_html):
    print(name.get_text())

Agaricus altipes
Agaricus arvensis
Agaricus augustus
Agaricus bernardii
Agaricus bisporus
Agaricus bitorquis
Agaricus campestris
Agaricus langei
Agaricus litoralis
Agaricus sylvaticus
Agaricus sylvicola
Agaricus urinascens
Agaricus xanthoderma
Agrocybe pediades
Agrocybe praecox
Albatrellus citrinus
Albatrellus confluens
Albatrellus ovinus
Albatrellus subrubescens
Albatrellus syringae
Amanita ceciliae
Amanita citrina
Amanita citrina f. alba
Amanita excelsa
Amanita gemmata
Amanita lividopallescens
Amanita muscaria
Amanita muscaria var. regalis
Amanita pantherina
Amanita phalloides
Amanita rubescens
Amanita virosa
Armillaria mellea s.lat.
Artomyces cristatus
Artomyces pyxidatus
Asterophora lycoperdoides
Aureoboletus gentilis
Bankera fuligineoalba
Boletopsis grisea
Boletus edulis
Boletus luridiformis var. discolor
Boletus pinophilus
Boletus reticulatus
Butyriboletus appendiculatus
Butyriboletus fechtneri
Caloboletus calopus
Caloboletus radicans
Calocybe gambosa
Cantharellula umbonata
Canth

In [7]:
print(len(latin_html))

377


NameError: name 'asjdakjsdh' is not defined